# Llama 2 Playground

The purpose of this notebook is to provide a place to mess around with the Llama 2 model.

In [1]:
%pip install -e ../submodules/llama

Obtaining file:///home/mila/c/caleb.moses/comp-550/assignment-1/submodules/llama
  Preparing metadata (setup.py) ... done
  Attempting uninstall: llama
    Found existing installation: llama 0.0.1
    Uninstalling llama-0.0.1:
      Successfully uninstalled llama-0.0.1
  Running setup.py develop for llama

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
from pathlib import Path

import yaml
import torch
from llama import Llama, Dialog

# Set environment variables
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12356'  # Choose any free port number
os.environ['RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"

ckpt_dir = '../models/llama-2-7b-chat'
tokenizer_path = '../models/llama/tokenizer.model'

max_seq_len = 512
max_batch_size = 8

generator = Llama.build(
    ckpt_dir=ckpt_dir,
    tokenizer_path=tokenizer_path,
    max_seq_len=max_seq_len,
    max_batch_size=max_batch_size,
)

dialogs = yaml.load(open('../dialogs/example.yaml'), Loader = yaml.FullLoader)

max_gen_len = None
temperature = 0.6
top_p = 0.9

results = generator.chat_completion(
        dialogs,  # type: ignore
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )

In [12]:
print(yaml.dump(results))

- generation:
    content: ' Mayonnaise is a thick, creamy condiment made from a mixture of egg
      yolks, oil, and an acid, such as vinegar or lemon juice. Here is a basic recipe
      for homemade mayonnaise:

      Ingredients:

      * 2 egg yolks

      * 1/2 cup (120 ml) neutral-tasting oil, such as canola or grapeseed

      * 1 tablespoon (15 ml) vinegar or lemon juice

      * Salt and pepper to taste

      Instructions:

      1. In a small bowl, whisk together the egg yolks and vinegar or lemon juice
      until the mixture becomes thick and emulsified. This should take about 5-7 minutes.

      2. Slowly pour the oil into the egg yolk mixture while continuously whisking.
      The mixture should start to thicken and emulsify as you add the oil.

      3. Continue whisking until the mixture becomes thick and creamy, and all of
      the oil has been incorporated. This can take up to 10-15 minutes.

      4. Taste and adjust the seasoning as needed. You may want to add mor

# Generating animal facts

Now that we've gotten the model to generate text, we can write our own dialogs to get it to compose new facts for us.

In [75]:
bird_names = ["Kiwi bird", "Tūī", "Kererū", "Kea", "Pūkeko", "Fantail (Pīwakawaka)", "Bellbird (Korimako)", "Tomtit (Miromiro)", "Rock Wren (Pīwauwau)", "New Zealand Falcon (Kārearea)"]

dialogs = []

for bird_name in bird_names:
    bird_dialog_yaml = open("../data/prompts/bird-facts.yaml").read()
    bird_dialog_yaml = bird_dialog_yaml.format(bird_name=bird_name)
    bird_dialog = yaml.load(bird_dialog_yaml, Loader=yaml.FullLoader)
    dialogs.extend(bird_dialog)

dialogs[0]

for dialog in dialogs:
    results = generator.chat_completion(
        [dialog],  # type: ignore
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )
    print(yaml.dump(results))

results = generator.chat_completion(
        bird_dialogs,  # type: ignore
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )

- generation:
    content: ' Of course! Here are 10 real facts about the Kiwi bird:

      FACT_START

      1. The Kiwi bird is the national bird of New Zealand and is known for its unique
      appearance, with a round body, small head, and long beak.

      FACT_END


      2. Kiwi birds have a highly developed sense of smell and can detect food from
      up to 20 meters away.

      FACT_START

      3. The Kiwi bird is nocturnal, meaning it is most active at night, and has excellent
      night vision.

      FACT_END


      4. Kiwi birds are solitary animals and only come together to mate.

      FACT_START

      5. The male Kiwi bird performs a complex courtship dance to attract a female,
      involving a series of struts, bobs, and wing flaps.

      FACT_END


      6. Kiwi birds lay a single egg per breeding season, and the female incubates
      the egg for about 20 days.

      FACT_START

      7. The chick hatches with its eyes open and is able to walk and run within 